In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix
# from scipy.sparse.linalg import eigh
import matplotlib.pyplot as plt
from itertools import combinations

In [5]:
# Physical parameters
N_unit_cells = 2       # Number of unit cells
v = 1                  # Intra-cell hopping
w = 2                  # Inter-cell hopping

# Derived quantities
N_orbitals = 2 * N_unit_cells  # Total fermionic orbitals (A and B per unit cell)
N_particles = N_unit_cells     # Half-filled system

In [6]:
def generate_basis(N_sites, N_particles):
    """Generate all Fock basis states with N_particles electrons."""
    basis = []
    for comb in combinations(range(N_sites), N_particles):
        state = [0] * N_sites
        for idx in comb:
            state[idx] = 1
        basis.append(tuple(state))
    return basis

basis = generate_basis(N_orbitals, N_particles)
print(basis)
# print(f"Dimension of Hilbert space: {len(basis)}")

[(1, 1, 0, 0), (1, 0, 1, 0), (1, 0, 0, 1), (0, 1, 1, 0), (0, 1, 0, 1), (0, 0, 1, 1)]


In [7]:
def fock_state_to_index(basis):
    """Map each Fock state to its index in the basis."""
    
    return {state: i for i, state in enumerate(basis)}

state_to_idx = fock_state_to_index(basis)
print(state_to_idx)

{(1, 1, 0, 0): 0, (1, 0, 1, 0): 1, (1, 0, 0, 1): 2, (0, 1, 1, 0): 3, (0, 1, 0, 1): 4, (0, 0, 1, 1): 5}


In [8]:
def apply_annihilation(state, i):
    if state[i] == 0:
        return None, 0
    new_state = list(state)
    new_state[i] = 0
    sign = 1
    for j in range(i):
        if state[j] == 1:
            sign *= -1
    return tuple(new_state), sign

def apply_creation(state, i):
    if state[i] == 1:
        return None, 0
    new_state = list(state)
    new_state[i] = 1
    sign = 1
    for j in range(i):
        if state[j] == 1:
            sign *= -1
    return tuple(new_state), sign

In [ ]:
from scipy.sparse import csr_matrix

def build_Hamiltonian(basis, N_unit_cells, v, w):
    dim = len(basis)
    H = csr_matrix((dim, dim), dtype=np.complex128)
    state_to_idx = fock_state_to_index(basis)

    for idx, state in enumerate(basis):
        # Intra-cell hopping: A <-> B
        for i in range(N_unit_cells):
            site_A = 2*i
            site_B = 2*i + 1

            # B -> A
            down_state, sign_down = apply_annihilation(state, site_B)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_A)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += v * sign_up * sign_down

            # A -> B (h.c.)
            down_state, sign_down = apply_annihilation(state, site_A)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_B)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += v * sign_up * sign_down

        # Inter-cell hopping: A_{i+1} <-> B_i
        for i in range(N_unit_cells - 1):
            site_B = 2*i + 1
            site_A_next = 2*(i+1)

            # B_i -> A_{i+1}
            down_state, sign_down = apply_annihilation(state, site_B)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_A_next)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += w * sign_up * sign_down

            # A_{i+1} -> B_i (h.c.)
            down_state, sign_down = apply_annihilation(state, site_A_next)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_B)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += w * sign_up * sign_down

    return H

H = build_Hamiltonian(basis, N_unit_cells, v, w)
print(H.toarray())

[[0.+0.j 2.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [2.+0.j 0.+0.j 1.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 1.+0.j 0.+0.j 2.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 2.+0.j 0.+0.j]]


In [19]:

eigenvalues, eigenvectors = np.linalg.eigh(H.toarray())

# Sort and print eigenvalues
print("Lowest eigenvalues:")
print(np.sort(eigenvalues))


Lowest eigenvalues:
[-2.82842712e+00 -2.00000000e+00  0.00000000e+00  6.41898740e-16
  2.00000000e+00  2.82842712e+00]


In [21]:
wf_ps = np.zeros((len(basis), len(basis[0])))

basis = np.array(basis)
for j, state in enumerate(eigenvectors):
    for i, weight in enumerate(state):
        wf_ps[j] += weight * basis[i]

UFuncTypeError: Cannot cast ufunc 'add' output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'